# Tg-Bot "Диетолог"

Ключ Моего GigaChat API: ZDA2NWU0YjItMjU5Zi00ZjBmLTg2NDItNmZkYWU4MjlkMGIyOjA1YTY1YjBlLWI1NWItNGMxZC1hMGUxLTFhNzVkMzNkZTg2ZA==

# СДЕЛАНО

In [ ]:
import logging
import time
import nest_asyncio
import sqlite3

from langchain_gigachat import GigaChat
from langchain_gigachat.chat_models import GigaChat
from langchain_core.messages import HumanMessage, SystemMessage


nest_asyncio.apply()

from telegram import ReplyKeyboardMarkup, ReplyKeyboardRemove, Update
from telegram.ext import (
    Application,
    CommandHandler,
    ContextTypes,
    ConversationHandler,
    MessageHandler,
    filters,
)

TG_BOT_TOKEN = "7742381241:AAH46TuiKZYsNWmGvJsdRO7a8BpaMuuk61Y"
GIGACHAT_API_KEY = "ZDA2NWU0YjItMjU5Zi00ZjBmLTg2NDItNmZkYWU4MjlkMGIyOjkzOGI4ZjRjLTQyMWQtNDY0OS1iY2Y2LTY1OGFmYTUwN2RiYQ=="

logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)

logging.getLogger("httpx").setLevel(logging.WARNING)

logger = logging.getLogger(__name__)

GENDER, AGE, HEIGTH, WEIGHT, DIET_TYPE, RECOMMENDATIONS = range(6)

conn = sqlite3.connect("user_data.db", check_same_thread=False)
cursor = conn.cursor()

# Создание таблицы для хранения данных пользователя
cursor.execute("""
CREATE TABLE IF NOT EXISTS user_data (
    user_id INTEGER PRIMARY KEY,
    gender TEXT,
    age INTEGER,
    height INTEGER,
    weight INTEGER,
    diet_type TEXT,
    gigachat_diet TEXT
)
""")

messages = [
    SystemMessage(
        content="Ты бот-диетолог, отвечающий на вопросы про еду и здоровье и помогающий людям с диетой."
    )
]

chat = GigaChat(credentials=GIGACHAT_API_KEY,verify_ssl_certs=False)

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user_id = update.effective_user.id
    cursor.execute("SELECT * FROM user_data WHERE user_id = ?", (user_id,))
    user_data = cursor.fetchone()

    if user_data:
        await update.message.reply_text(
            "Рады снова видеть вас!\n\n"
            "Напоминаем основной функционал нашего бота:\n\n"
            "/colories - Необходимое количество колорий, потребляемых за день\n"
            "/sleep - Необходимое количество часов сна\n"
            "/water - Необходимое количество воды, выпиваемой в течение дня\n"
            "/networkdiet - Сгенерировать диету под ваши данные, используя нейросеть GigaChat.\n"
            "/viewdiet - Просмотр диеты, соответствующая вашим данным, сгенерированная нейросетью\n"
            "——————————————————\n"
            "/cancel - Закончить диалог с ботом")
        return RECOMMENDATIONS
    else:
        reply_keyboard = [["Мужчина", "Женщина"]]
        await update.message.reply_text(
            "Привет, я Телеграмм бот 'Диетолог' Я помогу тебе следить за правильным питанием и также придерживаться диете, которую ты выберешь. " 
            "Для начала мне необходимо узнать твои данные.\n\n"
            "Введите свой пол", 
            parse_mode='HTML',
            reply_markup=ReplyKeyboardMarkup(
                reply_keyboard, one_time_keyboard=True, input_field_placeholder="Выберите пол"
            )   
        )

        return GENDER

async def gender(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    
    user = update.message.from_user
    user_gender = update.message.text
    logger.info("Gender of %s: %s", user.first_name, update.message.text)

    reply_keyboard = [["Мужчина", "Женщина"]]
    if (update.message.text != "Мужчина" and update.message.text != "Женщина"):

        await update.message.reply_text("Вы ввели некорректные данные.\nПожалуйста, выберите пол: 'Мужчина' или 'Женщина'",
        parse_mode='HTML',
        reply_markup=ReplyKeyboardMarkup(
            reply_keyboard, one_time_keyboard=True, input_field_placeholder="Выберите пол"
        ))
        
    if (update.message.text == '/colories'):
        await update.message.reply_text("Для начала необходимо завершить процесс регистрации!\nПожалуйста, выберите пол: 'Мужчина' или 'Женщина'",
        parse_mode='HTML',
        reply_markup=ReplyKeyboardMarkup(
            reply_keyboard, one_time_keyboard=True, input_field_placeholder="Выберите пол"
        ))

    if (update.message.text == "Мужчина" or update.message.text == "Женщина"):
        context.user_data['gender'] = user_gender    
        await update.message.reply_text(
        reply_markup=ReplyKeyboardRemove(),
        text="Отлично, сколько вам лет?",
        parse_mode="HTML"
        )
        return AGE 

async def age(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user = update.message.from_user
    logger.info("Age of %s: %s", user.first_name, update.message.text)

    try:
        if (10 <= int(update.message.text) <= 90):

            user_age = int(update.message.text)
            context.user_data['age'] = user_age

            await update.message.reply_text("Хорошо, теперь нужно узнать ваш рост и вес.\n\nДля начала введите ваш рост")
            return HEIGTH
        
        elif (int(update.message.text) < 10 or int(update.message.text) > 90):
            await update.message.reply_text("Вам не положено пользоваться нашим ботом в из-за возрастных ограничений.\n"
            "Если же вы по случайности ввели не тот возраст, введите ваш корректный возраст")

    except ValueError:
        await update.message.reply_text("Введен некорректный возраст.\n"
            "Ваш возраст должен быть целым число и находиться в диапазоне от 10 до 90 лет")

async def height(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user = update.message.from_user
    logger.info("Heaght of %s: %s", user.first_name, update.message.text)

    try:
        if (90 <= int(update.message.text) <= 260):

            user_height = int(update.message.text)
            context.user_data['height'] = user_height

            await update.message.reply_text("Так, а теперь введите свой вес")
            return WEIGHT

        elif (int(update.message.text) < 90 or int(update.message.text) > 260):
            await update.message.reply_text("К сожалению ваш вес не подходит под ограничения для нашего телеграмм бота.\n"
            "Если же вы по случайности ввели не то значение веса, введите ваш корректный вес")
    
    except ValueError:
        await update.message.reply_text("Введен некорректное значение роста.\n"
            "Ваш рост должен быть целым число и находиться в диапазоне от 90 до 260 см") 

async def weight(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    
    reply_keyboard = [["Снижение веса", "Поддержание веса", "Набор массы"]]
    user = update.message.from_user
    logger.info("Weight of %s: %s", user.first_name, update.message.text)

    try:
        if (35 <= int(update.message.text) <= 170):

            user_weight = int(update.message.text)
            context.user_data['weight'] = user_weight

            await update.message.reply_text(
                "Отлично, осталось только выбрать тип диеты. Выберите тип диеты...",
                reply_markup=ReplyKeyboardMarkup(
                    reply_keyboard, one_time_keyboard=True, input_field_placeholder="Выберите диету"))

            return DIET_TYPE

        elif (int(update.message.text) < 35 or int(update.message.text) > 170):
            await update.message.reply_text("К сожалению ваш вес не подходит под ограничения для нашего телеграмм бота.\n"
            "Если же вы по случайности ввели не то значение веса, введите ваш корректный вес")
    
    except ValueError:
        await update.message.reply_text("Введен некорректное значение веса.\n"
            "Ваш вес должен быть целым число и находиться в диапазоне от 35 до 170 кг") 

async def diet(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user = update.message.from_user
    logger.info("Diet type of %s: %s", user.first_name, update.message.text)

    user_diet_type = update.message.text.lower()
    
    reply_keyboard = [["Снижение веса", "Поддержание веса", "Набор массы"]]

    if (user_diet_type != "снижение веса" and user_diet_type != "поддержание веса" and user_diet_type != "набор массы"):
        await update.message.reply_text("Вы ввели некорректные данные.\nПожалуйста, выберите одну из предложенных диет: 'Снижение веса'| 'Поддержание веса' | 'Набор массы'",
        parse_mode='HTML',
        reply_markup=ReplyKeyboardMarkup(
            reply_keyboard, one_time_keyboard=True, input_field_placeholder="Выберите тип диеты"
        ))
        
    if (user_diet_type == "снижение веса" or user_diet_type == "поддержание веса" or user_diet_type == "набор массы"):
        context.user_data['diet_type'] = user_diet_type    

        await update.message.reply_text(
                "Прекрасно, сейчас загрузим необходимую информацию в базу данных...",
                reply_markup=ReplyKeyboardRemove(),
                parse_mode="HTML"
        )

        user_id = update.effective_user.id
        save_user_data(user_id, context.user_data['gender'], context.user_data['age'], context.user_data['height'], context.user_data['weight'], context.user_data['diet_type'])

        time.sleep(3)
        await update.message.reply_text(
            "Готово!\n\nТеперь вам доступен базовый функционал нашего телеграмм бота:\n\n"
            "/colories - Необходимое количество колорий, потребляемых за день\n"
            "/sleep - Необходимое количество часов сна\n"
            "/water - Необходимое количество воды, выпиваемой в течение дня\n"
            "/networkdiet - Сгенерировать диету под ваши данные, используя нейросеть GigaChat.\n"
            "/viewdiet - Просмотр диеты, соответствующая вашим данным, сгенерированная нейросетью\n"
            "——————————————————\n"
            "/cancel - Закончить диалог с ботом")


        return RECOMMENDATIONS
        

async def colories(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user_id = update.effective_user.id
    cursor.execute("SELECT gender, age, height, weight, diet_type FROM user_data WHERE user_id = ?", (user_id,))
    user_data = cursor.fetchone()

    if user_data:
        gender, age, height, weight, diet_type = user_data
        bmr = calculate_bmr(gender, weight, height, age)
        daily_calories = calculate_daily_calories(bmr, diet_type)
        message = generate_message(diet_type, daily_calories)
        await update.message.reply_text(message)
    else:
        await update.message.reply_text(
            "Не удалось получить данные пользователя.\n"
            "Сначала завершите регистрацию\n"
            )

    return RECOMMENDATIONS


def calculate_bmr(gender, weight, height, age):
    if gender == "Мужчина":
        return (88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * age))
    elif gender == "Женщина":
        return (447.593 + (9.247 * weight) + (3.098 * height) - (4.33 * age))


def calculate_daily_calories(bmr, diet_type):
    
    # Для простоты, предположим, что пользователь имеет среднюю активность
    activity_factor = 1.55

    if diet_type == "снижение веса":
        return bmr * activity_factor - 250  # Уменьшаем на 250 калорий для снижения веса
    elif diet_type == "поддержание веса":
        return bmr * activity_factor
    elif diet_type == "набор массы":
        return bmr * activity_factor + 250  # Увеличиваем на 250 калорий для набора массы


def generate_message(diet_type, daily_calories):
    if diet_type == "снижение веса":
        return f"Калорийность вашей диеты для снижения веса ≈ {round(daily_calories, 2)} ккал/день"
    elif diet_type == "поддержание веса":
        return f"Калорийность вашей диеты для поддержания веса ≈ {round(daily_calories, 2)} ккал/день"
    elif diet_type == "набор массы":
        return f"Калорийность вашей диеты для набора массы ≈ {round(daily_calories, 2)} ккал/день"

async def sleep(update:Update, context: ContextTypes.DEFAULT_TYPE) -> int:

    user_id = update.effective_user.id
    cursor.execute("SELECT gender, age, height, weight, diet_type FROM user_data WHERE user_id = ?", (user_id,))
    user_data = cursor.fetchone()

    if user_data:
        await update.message.reply_text(
        "Для ваших данных необходимо спать от 7 до 9 часов в сутки"
        )

    else:
        await update.message.reply_text(
            "Не удалось получить данные пользователя.\n"
            "Сначала завершите регистрацию\n"
            )
    
    return RECOMMENDATIONS

async def water(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user_id = update.effective_user.id
    cursor.execute("SELECT gender, age, height, weight, diet_type FROM user_data WHERE user_id = ?", (user_id,))
    user_data = cursor.fetchone()

    if user_data:
        gender, age, height, weight, diet_type = user_data
        recommended_weight = calculate_recommended_weight(height)
        daily_water = calculate_daily_water(recommended_weight, diet_type)
        message = generate_water_message(daily_water)
        await update.message.reply_text(message)
    else:
        await update.message.reply_text(
            "Не удалось получить данные пользователя.\n"
            "Сначала завершите регистрацию\n"
            )

    return RECOMMENDATIONS


def calculate_recommended_weight(height):
    # Простая формула для расчета рекомендуемого веса
    return height - 100


def calculate_daily_water(recommended_weight, diet_type):
    # Для нормального веса
    if diet_type == "поддержание веса":
        return recommended_weight * 30  # 30 мл на 1 кг рекомендуемого веса
    # Для снижения веса
    elif diet_type == "снижение веса":
        return recommended_weight * 40  # 40 мл на 1 кг рекомендуемого веса
    # Для набора массы
    elif diet_type == "набор массы":
        return recommended_weight * 30  # 30 мл на 1 кг рекомендуемого веса


def generate_water_message(daily_water):
    # Преобразование мл в литры
    daily_water_liters = daily_water / 1000
    pure_water_percentage = 0.6
    pure_water = daily_water_liters * pure_water_percentage

    message = f"Рекомендуемое количество воды для вас составляет примерно {daily_water_liters:.2f} литров в день.\n"
    message += f"Из этого объема желательно, чтобы {pure_water:.2f} литров была чистая питьевая вода."
    return message

async def network_diet(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user_id = update.effective_user.id
    cursor.execute("SELECT gender, age, height, weight, diet_type FROM user_data WHERE user_id = ?", (user_id,))
    user_data = cursor.fetchone()

    if user_data:
        gender, age, height, weight, diet_type = user_data
        
        user_message = f"Порекомендуй диету человеку пола {gender}, возраста {age} лет, с целью {diet_type} с весом {weight} кг и ростом {height} см"
        messages.append(HumanMessage(content=user_message))
        
        result = chat.invoke(messages)

        if isinstance(result, SystemMessage) or hasattr(result, "content"):
            await update.message.reply_text(result.content)
            messages.append(result)  # Добавляем ответ в историю

            # Save the generated diet to the database
            cursor.execute("""
                UPDATE user_data
                SET gigachat_diet = ?
                WHERE user_id = ?
            """, (result.content, user_id))
            conn.commit()
            logger.info("Diet saved successfully for user %s", user_id)
    else:
        await update.message.reply_text(
            "Не удалось получить данные пользователя.\n"
            "Сначала завершите регистрацию\n"
            )

    return RECOMMENDATIONS

async def view_saved_diet(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user_id = update.effective_user.id
    cursor.execute("SELECT gigachat_diet FROM user_data WHERE user_id = ?", (user_id,))
    saved_diet = cursor.fetchone()

    try:

        if saved_diet:
            await update.message.reply_text("Ваша сохраненная диета:\n" + saved_diet[0])
        else:
            await update.message.reply_text("Не удалось найти сохраненную диету.")

        return RECOMMENDATIONS
    
    except TypeError:
        await update.message.reply_text("Не удалось найти сохраненную диету. Для начала ее необходимо сгенерировать нейросетью")

async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user = update.message.from_user
    logger.info("User %s canceled the conversation.", user.first_name)
    await update.message.reply_text(
        "Пока! Надеюсь мы еще спишемся).", reply_markup=ReplyKeyboardRemove()
    )

    return ConversationHandler.END

def save_user_data(user_id, gender, age, height, weight, diet_type):
    try:
        cursor.execute('''
            INSERT OR REPLACE INTO user_data (user_id, gender, age, height, weight, diet_type)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (user_id, gender, age, height, weight, diet_type))
        conn.commit()
        logger.info("Data saved successfully for user %s", user_id)
    except sqlite3.Error as e:
        logger.error("Error saving data: %s", e)


def main() -> None:

    application = Application.builder().token(TG_BOT_TOKEN).build()

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler("start", start)],
        states={
            GENDER: [MessageHandler(filters.TEXT & ~filters.COMMAND,gender)],
            AGE: [MessageHandler(filters.TEXT & ~filters.COMMAND, age)],
            HEIGTH: [MessageHandler(filters.TEXT & ~filters.COMMAND, height)],
            WEIGHT: [MessageHandler(filters.TEXT & ~filters.COMMAND, weight)],
            DIET_TYPE: [MessageHandler(filters.TEXT & ~filters.COMMAND, diet)],
            RECOMMENDATIONS: [ 
            CommandHandler('colories',colories),
            CommandHandler('sleep', sleep),
            CommandHandler('water', water),
            CommandHandler('networkdiet', network_diet),
            CommandHandler('viewdiet', view_saved_diet)]
        },
        fallbacks=[CommandHandler("cancel", cancel)],
    )

    application.add_handler(conv_handler)

    application.run_polling(allowed_updates=Update.ALL_TYPES)


if __name__ == "__main__":
    main()

2024-12-11 15:52:19,854 - telegram.ext.Application - INFO - Application started
2024-12-11 15:52:35,225 - __main__ - INFO - Diet saved successfully for user 927622783


# GigaChat API
### Уже не нужно(тренировочный тест)

In [ ]:
#ZDA2NWU0YjItMjU5Zi00ZjBmLTg2NDItNmZkYWU4MjlkMGIyOjA1YTY1YjBlLWI1NWItNGMxZC1hMGUxLTFhNzVkMzNkZTg2ZA==

from langchain_gigachat.chat_models import GigaChat
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

sber = "ZDA2NWU0YjItMjU5Zi00ZjBmLTg2NDItNmZkYWU4MjlkMGIyOjA1YTY1YjBlLWI1NWItNGMxZC1hMGUxLTFhNzVkMzNkZTg2ZA=="

parser = StrOutputParser()

model = GigaChat(
    credentials=sber,
    scope="GIGACHAT_API_PERS",
    model="GigaChat",
    verify_ssl_certs=False,
)

system_template = "Ты бот-диетолог, отвечающий на вопросы про еду и здоровье и помогающий людям с диетой. Сгенерируй диету для {diet_type}. Вот данные о пользователе: Пол - {gender}, Возраст - {age}, Рост - {height}, Вес - {weight}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

# result = prompt_template.invoke({"diet_type": "Снижение веса", "gender": "Мужчина", "age": "40", "height": "180", "weight": "90", "text": "Привет, я бы хотел снизить вес. Я мужчина, мне 40 лет, мой рост 180, вес 90"})

chain = prompt_template | model | parser

answer = chain.invoke({"diet_type": "Снижение веса", "gender": "Мужчина", "age": "40", "height": "180", "weight": "90", "text": "Привет, я бы хотел снизить вес. Я мужчина, мне 40 лет, мой рост 180, вес 90"})
print(answer)

AIMessage(content='Здравствуйте! Всё отлично, готов помочь вам с любыми вопросами. Чем могу быть полезен?', additional_kwargs={}, response_metadata={'token_usage': Usage(prompt_tokens=19, completion_tokens=25, total_tokens=44), 'model_name': 'GigaChat:1.0.26.15', 'finish_reason': 'stop'}, id='run-c272e22d-20ad-4d21-ad40-be5bee68f0f4-0')

# SQLite3
### Уже не нужно(тренировочный тест)

In [ ]:
conn = sqlite3.connect("user_data.db", check_same_thread=False)
cursor = conn.cursor()

# Создание таблицы для хранения данных пользователя
cursor.execute("""
CREATE TABLE IF NOT EXISTS user_data (
    user_id INTEGER PRIMARY KEY,
    goal TEXT,
    age INTEGER,
    gender TEXT,
    weight INTEGER,
    height INTEGER
)
""")
conn.commit()